In [16]:
# Example of converting a Json format to a diagram

from json_ntv import Ntv

def diagram(diag):
    ntv = Ntv.obj(diag)
    diag_type = ntv.type_str[1:]
    diag_txt = '---\ntitle: ' + ntv.name + '\n---\n' if ntv.name else ''
    diag_txt += diag_type
    match diag_type:
        case 'erDiagram':
            diag_txt += erDiagram(ntv)
    return diag_txt
    
def erDiagram(ntv):
    diag_txt = ''
    for item in Ntv.obj(ntv.val):
        if item.name == 'entity':
            for entity in item:
                diag_txt += erEntity(entity)
        if item.name == 'relationship':
            for relation in item:
                diag_txt += erRelation(relation)
    return diag_txt

def erEntity(entity):
    ent_txt = '\n    ' + entity.name + ' {'
    for att in entity:
        ent_txt += '\n        ' + att[0].val + ' ' + att[1].val
        if len(att) > 2:
            if att[2].val in ('PK', 'FK', 'UK'):
                ent_txt += ' ' + att[2].val
            else:
                ent_txt += ' "' + att[2].val + '"'
        if len(att) > 3:
            ent_txt += ' "' + att[3].val + '"'
    return ent_txt + '\n    }'

def erRelation(rel):
    rel_left  = {'exactly one' : ' ||', 'zero or one': ' |o', 'zero or more': ' }o', 'one or more': ' }|'}
    rel_right = {'exactly one' : '|| ', 'zero or one': 'o| ', 'zero or more': 'o{ ', 'one or more': '|{ '}
    identif   = {'identifying' : '--', 'non-identifying' : '..'}
    rel_txt = '\n    ' + rel[0].val + rel_left[rel[1].val] + identif[rel[2].val] + rel_right[rel[3].val] + rel[4].val
    if len(rel) > 5:
        rel_txt += ' : ' + rel[5].val
    return rel_txt

In [17]:
# The Json syntax is enriched by separator ':' or '::' in dict key to indicate that:
#  - the second part of the key string is the data type
#  - the dict value is a json entity (':') or a list of Json-NTV entities.
order_example = { 
    'order example:$erDiagram' : { 
        'relationship::': [ 
            [ 'CUSTOMER', 'exactly one', 'identifying', 'zero or more', 'ORDER',     'places'],
            [ 'ORDER',    'exactly one', 'identifying', 'one or more',  'LINE-ITEM', 'contains'] 
        ],
        'entity::': {
            'CUSTOMER':  [ 
                ['string', 'name',         'PK', 'the name'], 
                ['string', 'custNumber'] 
            ], 
            'ORDER': [ 
                ['int',    'orderNumber',  'PK'],
                ['string', 'deliveryAdress'] 
            ],
            'LINE-ITEM': [ 
                ['string', 'productCode',  'PK'],
                ['int',    'quantity'],
                ['float',  'pricePerUnit'] 
            ]   
        },
     } }

In [18]:
from base64 import b64encode
from IPython.display import Image, display

diag = diagram(order_example)
display(Image(url="https://mermaid.ink/img/" + b64encode(diag.encode("ascii")).decode("ascii")))